In [2]:
#*************************************************************************
# Talha Shahzad
# 21i-0540
# Section: C
# Final Project
# Artificial Intelligence
#*************************************************************************

In [4]:
Course=['OS','Algo','DB','DS','PF','OOP','CAL','AP','LA','AI','STATS','MLOPS','IS','SE','SDA']
status=['Lab','Theory']
Section=['A','B','C']
Professor=['T1','T2','T3','T4','T5']
First_lecture_timeslot=['8:30-9:50','10:05-11:25','11:40-1:00','1:15-2:35','4:00-5:30']
First_lecture_room_size=['classroom','hall']


In [ ]:
import random
Courses = ["Programming Fundamentals", "Data Structures", "Algorithms", "Database Management Systems", "Operating Systems", "Computer Networks", "Software Engineering", "Web Engineering", "Mobile Application Development", "Artificial Intelligence", "Machine Learning", "Data Science", "Internet of Things", "Cloud Computing", "Cyber Security", "Network Security", "Discrete Mathematics", "Linear Algebra", "Calculus", "Applied Physics", "Computer Assembly", "English", "Islamiat", "Fundamentals of Marketing", "Marketing Management", "Psychology", "Web", "SMD", "DIP", "Advanced Programming", "Data Science", "Machine Learning", "AI", "IoT", "Cloud Computing", "Cyber Security", "Network Security"]
Teachers = ["Teacher 1", "Teacher 2", "Teacher 3", "Teacher 4", "Teacher 5", "Teacher 6", "Teacher 7", "Teacher 8", "Teacher 9", "Teacher 10", "Teacher 11", "Teacher 12", "Teacher 13", "Teacher 14", "Teacher 15"]
Sections = ["Section 1", "Section 2", "Section 3", "Section 4", "Section 5"]
Floors = 5
Classrooms = 10
Labs = 3
Classes = []

for i in range(Floors):
    temp= []
    for j in range(Classrooms):
        temp.append("Floor {} Room {} Capacity {}".format(i+1, j+1,60))
    Classes.append(temp)


for i in range(Floors):
    temp= []
    for j in range(Labs):
        temp.append("Floor {} Lab {} Capacity {}".format(i+1, j+1,120))
    Classes.append(temp)

# randomly making the capacity of 5 rooms 120
for i in range(5):
    Classes[i][random.randint(0,9)] = "Floor {} Room {} Capacity {}".format(i+1, random.randint(0,9)+1, 120)

# randomly making the capacity of 3 labs 60
for i in range(5,8):
    Classes[i][random.randint(0,2)] = "Floor {} Lab {} Capacity {}".format(i-4, random.randint(0,2)+1, 60)

import datetime as dt
TimeSlots = []
BreaskTime = 10
ClassDuration = 80
StartTime = dt.datetime.strptime("08:30", "%H:%M")
TotalClassesPerDay = 7
for i in range(TotalClassesPerDay):
    TimeSlots.append((StartTime + dt.timedelta(minutes=(BreaskTime + ClassDuration) * i)).strftime("%H:%M"))

Days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

# Creating a random timetable Each day has 7 time slots, each time slot has classes in every classroom and lab of every floor has a random course, teacher, and section
def TimeTableGenrator():
    TimeTable = {}
    for day in Days:
        for time_slot in TimeSlots:
            TimeTable[(day, time_slot)] = []
            for section in Sections:
                course = random.choice(Courses)
                room = random.choice(Classes[random.randint(0,7)])
                teacher = random.choice(Teachers)
                TimeTable[(day, time_slot)].append({
                    "section": section,
                    "course": course,
                    "room": room,
                    "teacher": teacher,
                    "type": "Lab" if "Lab" in room else "Theory"
                })
    return TimeTable

# generating chromosones in the form of a list with format
# Course, Theory/Lab, Section, Section-Strength, Professor, First-lecture-day,First- lecture-timeslot, First-lecture-room, First-lecture-room-size, Second-lecture-day, Second-lecture-timeslot, Second-lecture-room, Second-lecture-room-size
# for each course
def generate_chromosomes(TimeTable):
    chromosomes = []
    for key, value in TimeTable.items():
        for val in value:
            if val["type"] == "Theory":
                room_size = 60
            else:
                room_size = 120
            chromosomes.append([val["course"], val["type"], val["section"], 30, val["teacher"], key[0], key[1], val["room"], room_size, key[0], key[1], val["room"], room_size])

    for i in chromosomes: # creating a hash value for each value shorted in the chromoses
        hash = 0
        for j in i:
            if isinstance(j, str):
                for char in j:
                    hash += ord(char)
            elif isinstance(j, int):
                hash += j
        chromosomes[chromosomes.index(i)].append(hash)

    return chromosomes

def Fitness(chromosomes):
    fitness = 0
    # Hard constraints
    # Classes can only be scheduled in free classrooms.
    # A classroom should be big enough to accommodate the section. There should be two categories of classrooms: classroom (60) and large hall (120).
    # A professor should not be assigned two different lectures at the same time.
    # The same section cannot be assigned to two different rooms at the same time.
    # A room cannot be assigned for two different sections at the same time.
    # No professor can teach more than 3 courses.
    # No section can have more than 5 courses in a semester.
    # Each course would have two lectures per week not on the same or adjacent days.
    # Lab lectures should be conducted in two consecutive slots.
    for i in range(len(chromosomes)):
        for j in range(i + 1, len(chromosomes)):
            if len(chromosomes[i]) > 7 and len(chromosomes[j]) > 7:  # Check if the length of the string is greater than or equal to 8
                if chromosomes[i][7] == chromosomes[j][7]:  # A room cannot be assigned for two different sections at the same time.
                    fitness += 1
            if chromosomes[i][8] < chromosomes[i][3]:  # A classroom should be big enough to accommodate the section. There should be two categories of classrooms: classroom (60) and large hall (120).
                fitness += 1
            if chromosomes[i][4] == chromosomes[j][4]:  # A professor should not be assigned two different lectures at the same time.
                fitness += 1
            if chromosomes[i][2] == chromosomes[j][2]:  # The same section cannot be assigned to two different rooms at the same time.
                fitness += 1

    for teacher in Teachers:
        courses = [chromosome[0] for chromosome in chromosomes if chromosome[4] == teacher] # No professor can teach more than 3 courses.
        if len(set(courses)) > 3:
            fitness += 1
    for section in Sections:
        courses = [chromosome[0] for chromosome in chromosomes if chromosome[2] == section] # No section can have more than 5 courses in a semester.
        if len(set(courses)) > 5:
            fitness += 1
    for section in Sections: # Each course would have two lectures per week not on the same or adjacent days.
        for course in Courses: # Each course would have two lectures per week not on the same or adjacent days.
            count = 0
            for chromosome in chromosomes:
                if chromosome[2] == section and chromosome[0] == course: # Each course would have two lectures per week not on the same or adjacent days.
                    count += 1
            if count > 2:   # Each course would have two lectures per week not on the same or adjacent days.
                fitness += 1
            if count < 2:  # Each course would have two lectures per week not on the same or adjacent days.
                fitness += 1
    for i in range(len(chromosomes)-1):
        if chromosomes[i][1] == "Lab" and chromosomes[i+1][1] == "Lab": # Lab lectures should be conducted in two consecutive slots.
            if chromosomes[i][6] != chromosomes[i+1][6] or chromosomes[i][7] != chromosomes[i+1][7]: # Lab lectures should be conducted in two consecutive slots.
                fitness += 1
    return (-1)*fitness

# Making Parent Selection,Two point Corossover and Mutation function for Genetic Algorithm to be run on the chromosomes
def ParentSelection(chromosomes):
    # generating 15 timetables and selecting the best 2
    TimeTables = []
    for i in range(15):
        TimeTable = TimeTableGenrator()
        TimeTables.append(TimeTable)

    Fitnesses = []
    for TimeTable in TimeTables:
        chromosomes = generate_chromosomes(TimeTable)
        Fitnesses.append(Fitness(chromosomes))
    best1 = TimeTables[Fitnesses.index(max(Fitnesses))]
    Fitnesses[Fitnesses.index(max(Fitnesses))] = -1
    best2 = TimeTables[Fitnesses.index(max(Fitnesses))]
    return best1, best2

def TwoPointCrossover(parent1, parent2):
    # the parents are converted to chromosomes
    chromosomes1 = generate_chromosomes(parent1)
    chromosomes2 = generate_chromosomes(parent2)
    # 2 point crossover is performed
    crossover_point1 = random.randint(0, len(chromosomes1)-1)
    crossover_point2 = random.randint(0, len(chromosomes1)-1)
    if crossover_point1 > crossover_point2:
        crossover_point1, crossover_point2 = crossover_point2, crossover_point1
    child1 = chromosomes1[:crossover_point1] + chromosomes2[crossover_point1:crossover_point2] + chromosomes1[crossover_point2:]
    child2 = chromosomes2[:crossover_point1] + chromosomes1[crossover_point1:crossover_point2] + chromosomes2[crossover_point2:]
    return child1, child2

def Mutation(chromosome):
    # randomly changing the course, teacher, room, and section of a chromosome
    mutation_point = random.randint(0, len(chromosome)-1)

    chromosome[mutation_point][0] = random.choice(Courses)
    chromosome[mutation_point][2] = random.choice(Sections)
    chromosome[mutation_point][4] = random.choice(Teachers)

    return chromosome


def DisplayChromoes(chromosomes):
    for chromosome in chromosomes:
        # printing all the data in the chromosome exclusing the last index rather displaying that in binary form
        print(chromosome[:-1],bin(chromosome[-1]))


# Running the Genetic Algorithm
# the fitness function takes in the entore time table as a list of chromosomes
# the chromosomes are generated from the timetable
# the chromosomes are selected based on the fitness function
# the chromosomes are crossed over
# the chromosomes are mutated
# the process is repeated for a number of generations
# the best chromosomes are selected
# the best chromosomes are converted back to the timetable
# the timetable is printed
TimeTable = TimeTableGenrator()
chromosomes = generate_chromosomes(TimeTable)

DisplayChromoes(chromosomes)

for generation in range(10):
    new_chromosomes = []
    for i in range(len(chromosomes)//2):
        parent1, parent2 = ParentSelection(chromosomes)
        child1, child2 = TwoPointCrossover(parent1, parent2)
        new_chromosomes.append(Mutation(child1))
        new_chromosomes.append(Mutation(child2))
    chromosomes = new_chromosomes
    print("Generation:", generation, "Fitness:", Fitness(chromosomes))

# finding max fitness of the chromosomes and saving index and printing the timetable
fitness = []
print(chromosomes)
for i in range(len(chromosomes)):
    # print(chromosomes[i])
    fitness.append(Fitness(chromosomes[i]))
index = fitness.index(max(fitness))
print(chromosomes[index])